In [1]:
import numpy as np
import pandas as pd
import fastparquet

In [2]:
trdf = pd.read_parquet("optiver/trade_train.parquet", engine = 'fastparquet')

trdf.stock_id = trdf.stock_id.astype(int)

len(trdf.query("""stock_id > 126"""))

0

In [3]:
bkdf = pd.read_parquet('optiver/book_train.parquet', engine = 'fastparquet').reset_index()

bkdf.stock_id = bkdf.stock_id.astype(int)

len(bkdf.query("""stock_id > 126"""))

0

In [4]:
"""kernel dead

tmpdf = pd.merge(trdf, bkdf, how='inner', on = ['stock_id', 'time_id','seconds_in_bucket'])


bkdf['index']+=1

df = pd.merge(tmpdf, bkdf, how='inner', on = ['index','stock_id','time_id'])



trdf0 = trdf.head(10000)
bkdf0 = bkdf.head(100000)

df0 = pd.merge(trdf0, bkdf0, how='inner', on = ['stock_id', 'time_id','seconds_in_bucket'])

bkdf0['index'] += 1

df1 = pd.merge(df0, bkdf0, how='inner', on = ['index','stock_id','time_id'])

df1

trdf0_s = trdf0.loc[:, ['time_id', 'seconds_in_bucket', 'stock_id']].reset_index()
trdf0_s.head()

bkdf0_s = bkdf0.loc[:, ['time_id', 'seconds_in_bucket', 'stock_id']].reset_index()
bkdf0_s.head()

df0_s = pd.merge(trdf0_s, bkdf0_s, how='inner', on=['time_id', 'stock_id', 'seconds_in_bucket'])
df0_s.head()

index_trdf0_s = df0_s.index_x

index_bkdf0_s_after = df0_s.index_y

index_bkdf0_s_before = index_bkdf0_s_after - 1

bk_before = bkdf0.iloc[index_bkdf0_s_before]
bk_before.head()

bk_after = bkdf0.iloc[index_bkdf0_s_after]
bk_after.head()

df_meow = pd.concat(
    [
        trdf0.iloc[index_trdf0_s].reset_index(),
        bk_before.reset_index(),
        bk_after.reset_index()
    ],
    axis=1
)
df_meow.head()

print(df_meow.columns)
"""

"kernel dead\n\ntmpdf = pd.merge(trdf, bkdf, how='inner', on = ['stock_id', 'time_id','seconds_in_bucket'])\n\n\nbkdf['index']+=1\n\ndf = pd.merge(tmpdf, bkdf, how='inner', on = ['index','stock_id','time_id'])\n\n\n\ntrdf0 = trdf.head(10000)\nbkdf0 = bkdf.head(100000)\n\ndf0 = pd.merge(trdf0, bkdf0, how='inner', on = ['stock_id', 'time_id','seconds_in_bucket'])\n\nbkdf0['index'] += 1\n\ndf1 = pd.merge(df0, bkdf0, how='inner', on = ['index','stock_id','time_id'])\n\ndf1\n\ntrdf0_s = trdf0.loc[:, ['time_id', 'seconds_in_bucket', 'stock_id']].reset_index()\ntrdf0_s.head()\n\nbkdf0_s = bkdf0.loc[:, ['time_id', 'seconds_in_bucket', 'stock_id']].reset_index()\nbkdf0_s.head()\n\ndf0_s = pd.merge(trdf0_s, bkdf0_s, how='inner', on=['time_id', 'stock_id', 'seconds_in_bucket'])\ndf0_s.head()\n\nindex_trdf0_s = df0_s.index_x\n\nindex_bkdf0_s_after = df0_s.index_y\n\nindex_bkdf0_s_before = index_bkdf0_s_after - 1\n\nbk_before = bkdf0.iloc[index_bkdf0_s_before]\nbk_before.head()\n\nbk_after = bkdf0.

In [5]:
def create_full_transaction_df(trdf, bkdf):
    match_cols = ['time_id', 'stock_id', 'seconds_in_bucket']
    trdf_match = trdf.loc[:, match_cols].reset_index()
    bkdf_match = bkdf.loc[:, match_cols].reset_index()
    df_index = pd.merge(trdf_match, bkdf_match, how='inner', on=match_cols)
    index_tr = df_index.index_x
    index_bk = df_index.index_y
    return index_tr, index_bk

In [6]:
index_tr, index_bk = create_full_transaction_df(trdf, bkdf)

In [7]:
index_tr.head()

0    0
1    1
2    2
3    3
4    4
Name: index_x, dtype: int64

In [8]:
index_bk.head()

0    13
1    17
2    20
3    23
4    27
Name: index_y, dtype: int64

In [9]:
trdf_ = trdf.iloc[index_tr, :].reset_index()
bkdf_ = bkdf.iloc[index_bk, :].reset_index()

In [10]:
df = pd.concat([trdf_, bkdf_], axis=1)
df.head()

,index,time_id,seconds_in_bucket,price,size,order_count,stock_id,level_0,index,time_id,seconds_in_bucket,bid_price1,ask_price1,bid_price2,ask_price2,bid_size1,ask_size1,bid_size2,ask_size2,stock_id
0,0,5,21,1.002301,326,12,0,13,13,5,21,1.001422,1.002818,1.001370,1.002922,3,30,2,100,0
1,1,5,46,1.002778,128,4,0,17,17,5,46,1.002818,1.003232,1.002301,1.003801,155,1,200,34,0
2,2,5,50,1.002818,55,1,0,20,20,5,50,1.002353,1.003025,1.002301,1.003232,3,20,100,301,0
3,3,5,57,1.003155,121,5,0,23,23,5,57,1.002508,1.003646,1.002457,1.003749,200,104,28,200,0
4,4,5,68,1.003646,4,1,0,27,27,5,68,1.002870,1.003749,1.002818,1.003801,100,100,11,34,0


In [11]:
df.drop(['index'], axis=1, inplace=True)

In [12]:
df.head()

,time_id,seconds_in_bucket,price,size,order_count,stock_id,level_0,time_id,seconds_in_bucket,bid_price1,ask_price1,bid_price2,ask_price2,bid_size1,ask_size1,bid_size2,ask_size2,stock_id
0,5,21,1.002301,326,12,0,13,5,21,1.001422,1.002818,1.001370,1.002922,3,30,2,100,0
1,5,46,1.002778,128,4,0,17,5,46,1.002818,1.003232,1.002301,1.003801,155,1,200,34,0
2,5,50,1.002818,55,1,0,20,5,50,1.002353,1.003025,1.002301,1.003232,3,20,100,301,0
3,5,57,1.003155,121,5,0,23,5,57,1.002508,1.003646,1.002457,1.003749,200,104,28,200,0
4,5,68,1.003646,4,1,0,27,5,68,1.002870,1.003749,1.002818,1.003801,100,100,11,34,0


In [13]:
df = df.loc[:, ~df.columns.duplicated()]
df.head()

,time_id,seconds_in_bucket,price,size,order_count,stock_id,level_0,bid_price1,ask_price1,bid_price2,ask_price2,bid_size1,ask_size1,bid_size2,ask_size2
0,5,21,1.002301,326,12,0,13,1.001422,1.002818,1.001370,1.002922,3,30,2,100
1,5,46,1.002778,128,4,0,17,1.002818,1.003232,1.002301,1.003801,155,1,200,34
2,5,50,1.002818,55,1,0,20,1.002353,1.003025,1.002301,1.003232,3,20,100,301
3,5,57,1.003155,121,5,0,23,1.002508,1.003646,1.002457,1.003749,200,104,28,200
4,5,68,1.003646,4,1,0,27,1.002870,1.003749,1.002818,1.003801,100,100,11,34


In [14]:
bkdf2 = bkdf.iloc[index_bk - 1, :]

In [15]:
bkdf2.reset_index(inplace=True)

In [16]:
bkdf2 = bkdf2.drop(['level_0', 'index'], axis=1)

In [17]:
def change_name_to_before(s: str):
    return s + '_before'

In [18]:
print(list(bkdf2.columns))

['time_id', 'seconds_in_bucket', 'bid_price1', 'ask_price1', 'bid_price2', 'ask_price2', 'bid_size1', 'ask_size1', 'bid_size2', 'ask_size2', 'stock_id']


In [19]:
bkdf2 = bkdf2.rename(columns={old_name : change_name_to_before(old_name) for old_name in bkdf2.columns})

In [20]:
bkdf2.head()

,time_id_before,seconds_in_bucket_before,bid_price1_before,ask_price1_before,bid_price2_before,ask_price2_before,bid_size1_before,ask_size1_before,bid_size2_before,ask_size2_before,stock_id_before
0,5,19,1.001422,1.002301,1.001370,1.002405,3,126,2,100,0
1,5,44,1.001732,1.002818,1.001422,1.002922,83,28,3,100,0
2,5,48,1.002818,1.003232,1.002353,1.003749,55,101,103,100,0
3,5,55,1.002353,1.003025,1.002301,1.003181,3,20,100,100,0
4,5,63,1.002818,1.003646,1.002767,1.003749,11,4,14,200,0


In [21]:
df = pd.concat([df, bkdf2], axis=1)

In [22]:
df.head()

,time_id,seconds_in_bucket,price,size,order_count,stock_id,level_0,bid_price1,ask_price1,bid_price2,...,seconds_in_bucket_before,bid_price1_before,ask_price1_before,bid_price2_before,ask_price2_before,bid_size1_before,ask_size1_before,bid_size2_before,ask_size2_before,stock_id_before
0,5,21,1.002301,326,12,0,13,1.001422,1.002818,1.001370,...,19,1.001422,1.002301,1.001370,1.002405,3,126,2,100,0
1,5,46,1.002778,128,4,0,17,1.002818,1.003232,1.002301,...,44,1.001732,1.002818,1.001422,1.002922,83,28,3,100,0
2,5,50,1.002818,55,1,0,20,1.002353,1.003025,1.002301,...,48,1.002818,1.003232,1.002353,1.003749,55,101,103,100,0
3,5,57,1.003155,121,5,0,23,1.002508,1.003646,1.002457,...,55,1.002353,1.003025,1.002301,1.003181,3,20,100,100,0
4,5,68,1.003646,4,1,0,27,1.002870,1.003749,1.002818,...,63,1.002818,1.003646,1.002767,1.003749,11,4,14,200,0


In [23]:
df = df.query("""stock_id == stock_id_before""")

_IncompleteInputError: incomplete input (3318514352.py, line 2)

In [26]:
df.to_parquet("meow_df.parquet",engine = 'fastparquet')

array([  0,   1,  10, 100, 101, 102, 103, 104, 105, 107, 108, 109,  11,
       110, 111, 112, 113, 114, 115, 116, 118, 119, 120, 122, 123, 124,
       125, 126,  13,  14,  15,  16,  17,  18,  19,   2,  20,  21,  22,
        23,  26,  27,  28,  29,   3,  30,  31,  32,  33,  34,  35,  36,
        37,  38,  39,   4,  40,  41,  42,  43,  44,  46,  47,  48,   5,
        50,  51,  52,  53,  55,  56,  58,  59,   6,  60,  61,  62,  63,
        64,  66,  67,  68,  69,   7,  70,  72,  73,  74,  75,  76,  77,
        78,   8,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,   9,
        90,  93,  94,  95,  96,  97,  98,  99])

In [1]:
# meow meow